In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# Make crime df for 2014 - 2024

In [2]:
kaggle_df = pd.read_csv('data/UK_Police_Street_Crime_2018-10-01_to_2021_09_31.csv')
crime_df = pd.read_csv('data/curated_data/crime.csv')
crime_cols = crime_df.columns
kaggle_df = kaggle_df[crime_cols]
kaggle_df = kaggle_df[kaggle_df['Reported by'] == 'Metropolitan Police Service']

In [14]:
data2021_2023 = pd.read_csv('data/curated_data/crime.csv')
data2018_2021 = kaggle_df
data2014_2017 = pd.read_csv('data/archive 3/london-street.csv')

In [19]:
crimes_total = data2021_2023.append(data2018_2021).append(data2014_2017)
crimes_total['year'] = crimes_total['Month'].apply(lambda x: int(x.split('-')[0]))
crimes_total['month'] = crimes_total['Month'].apply(lambda x: int(x.split('-')[1]))
crimes_total.drop(columns=['Month', 'Longitude', 'Latitude', 'Context'], inplace=True)
crimes_total = crimes_total[crimes_total['Reported by'] == 'Metropolitan Police Service']

/var/folders/vw/rm1q12851jq6ww7xsgf4g0x00000gn/T/ipykernel_88969/2833359830.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crimes_total = data2021_2023.append(data2018_2021).append(data2014_2017)
/var/folders/vw/rm1q12851jq6ww7xsgf4g0x00000gn/T/ipykernel_88969/2833359830.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crimes_total = data2021_2023.append(data2018_2021).append(data2014_2017)


# Make outcomes df for 2014 - 2024

In [43]:
outcomes21_23 = pd.read_csv('data/curated_data/outcome.csv')
outcomes14_17 = pd.read_csv('data/archive 3/london-outcomes.csv')
outcomes_total = outcomes21_23.append(outcomes14_17)
outcomes_total['year'] = outcomes_total['Month'].apply(lambda x: int(x.split('-')[0]))
outcomes_total['month'] = outcomes_total['Month'].apply(lambda x: int(x.split('-')[1]))
outcomes_total.drop(columns=['Month'], inplace=True)
outcomes_total = outcomes_total[outcomes_total['Reported by'] == 'Metropolitan Police Service']

# Merge crimes and outcomes

In [58]:
crimes_outcomes = crimes_total.merge(outcomes_total, on='Crime ID', how='left')

##### WEIRD THING, length of crimes_outcomes is larger than crimes

In [56]:
x = crimes_outcomes[crimes_outcomes['Crime ID'] == '65b197b2e13328e3a4e2a7cf8002a9051f0024fa4e22cd2abc8b961c9d131928']

In [70]:
mycounts = crimes_outcomes['Crime ID'].value_counts()
len(mycounts.loc[mycounts > 1])

1112763

In [55]:
x['Crime ID'].value_counts()

65b197b2e13328e3a4e2a7cf8002a9051f0024fa4e22cd2abc8b961c9d131928    8
Name: Crime ID, dtype: int64

In [73]:
crimes_outcomes.to_csv('data/curated_data/crimes_outcomes2014_2023.csv', index=False)

In [76]:
crimes_outcomes.columns

Index(['Crime ID', 'Reported by_x', 'Falls within_x', 'Location_x',
       'LSOA code_x', 'LSOA name_x', 'Crime type', 'Last outcome category',
       'Context', 'year_x', 'month_x', 'Reported by_y', 'Falls within_y',
       'Longitude', 'Latitude', 'Location_y', 'LSOA code_y', 'LSOA name_y',
       'Outcome type', 'year_y', 'month_y'],
      dtype='object')

### Adding information about the resolved/unresolved case

In [79]:
crime_outcomes = {
    "Investigation complete; no suspect identified": "unresolved",
    "Suspect charged": "resolved",
    "Local resolution": "resolved",
    "Offender given a caution": "resolved",
    "Offender given a drugs possession warning": "resolved",
    "Offender given penalty notice": "resolved",
    "Offender sent to prison": "resolved",
    "Defendant found not guilty": "resolved",
    "Offender given community sentence": "resolved",
    "Offender fined": "resolved",
    "Offender given suspended prison sentence": "resolved",
    "Offender given conditional discharge": "resolved",
    "Court case unable to proceed": "unresolved",
    "Offender otherwise dealt with": "resolved",
    "Offender ordered to pay compensation": "resolved",
    "Offender deprived of property": "resolved",
    "Suspect charged as part of another case": "resolved",
    "Offender given absolute discharge": "resolved",
    "Defendant sent to Crown Court": "resolved",
    "Formal action is not in the public interest": "resolved",
    "Unable to prosecute suspect": "unresolved"
}


crimes_outcomes['outcome'] = crimes_outcomes['Outcome type'].apply(lambda x: crime_outcomes[x] if not pd.isna(x)  else x)

In [81]:
crimes_outcomes['outcome'].value_counts()

unresolved    4207450
resolved       878448
Name: outcome, dtype: int64

# Merger locations